In [22]:
import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras import metrics
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.applications import mobilenet_v2
from keras.metrics import confusion_matrix
# from keras.applications import resnet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer

from keras_visualizer import visualizer 


ImportError: cannot import name 'confusion_matrix' from 'keras.metrics' (/Applications/anaconda3/lib/python3.8/site-packages/keras/metrics.py)

In [3]:
df_image_MoA = pd.read_csv('df_image_MoA.csv')
df_image_MoA.head()

,Unnamed: 0,Image,Drug,Compound MoA,Compound Name
0,0,11340528,Huperzine A,NaN,Huperzine A
1,1,11340529,Huperzine A,NaN,Huperzine A
2,2,11340530,Huperzine A,NaN,Huperzine A
3,3,11340525,nalbuphine,GPCR agonist,nalbuphine
4,4,11340526,nalbuphine,GPCR agonist,nalbuphine


In [5]:
df_image_MoA['Compound MoA'].value_counts().head()

Kinase inhibitor       2930
Chromatin structure    1782
GPCR agonist           1641
GPCR antagonist        1437
Channel blocker        1161
Name: Compound MoA, dtype: int64

## Set up data input

### Prepare images by converting PNGs to arrays

In [ ]:
def prepare_image(img_path):
    """ Takes in an image number, loads the png, and converts to array"""
    try:        
        img = image.load_img('raw_images/'+str(img_path)+'.png', target_size=(224, 224))
    except: 
        print(img_path)
    else:
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = mobilenet_v2.preprocess_input(x)
        return x

In [ ]:
# Get list of images
image_list = list(df_image_MoA.Image)

# Loop through images and collect processed arrays into list
array_list = []
for image_ in image_list:
    array_list.append(prepare_image(image_))
    


In [ ]:
image_array = np.vstack(array_list[0:25998])

In [ ]:
np.save('image_array', image_array, allow_pickle = True)

In [6]:
image_array = np.load('image_array.npy')

### Define the y-target (mechanism of action) of interest and create train-val-test splits

Methods used for test-train split
1. sklearn train-test split, Keras validation split
    - Keras only takes the last part of data, does not shuffle
2. sklearn train-test split, sklearrn train-validation split
    - Uneven number of target and non-target class
3. Stratify the train-test split and train-validation split
    - Equal number of target and non-target class 

Methods used for category encodin
1. One-hot encoding using np_utils.to_categorical
    - Creates an (x,2) array 
    - Resulted in having an equal number of true positives and true negatives in the validation data. 
    - True positives + false positives = total data set
2. 

In [7]:
def get_MoA(MoA):
    """Returns train-test split stratified on target class(MoA)"""    
    df_image_MoA['Category'] = np.where(df_image_MoA['Compound MoA'] == MoA, 1, 0)
    
    # Assign X and y 
    X_ = image_array
    y_ = df_image_MoA['Category'][0:25998]

    # Train-test split
    X_train_, X_test, y_train_, y_test = (train_test_split(X_, y_, 
                                                     test_size = .2, random_state = 22, stratify = y_))
    y_test = y_test.astype('float32')
    
    X_train, X_valid, y_train, y_valid = (train_test_split(X_train_, y_train_, 
                                                     test_size = .25, random_state = 22, stratify = y_train_))
    
    y_train = y_train.astype('float32')
    y_valid = y_valid.astype('float32')
    return X_train, y_train, X_test, y_test, X_valid, y_valid

In [8]:
X_train, y_train_cat, X_test, y_test_cat, X_valid, y_valid_cat = get_MoA('Chromatin structure')

Check the shape of each split

In [9]:
# Need to convert y labels from series to numpy arrays
y_test_array = y_test_cat.to_numpy()
y_train_array = y_train_cat.to_numpy()
y_valid_array = y_valid_cat.to_numpy()

## Test different hyperparameters on a CNN

### First set of overnight experiments
#### CNN models from scratch
1. NN_scratch - initial attempt
    - Did not do better than chance, overfit immediately
2. NN_scratch1 - add convolution layer
    - Did not do better than chance
3. NN_scratch2 - add dense layer 
    - Did not do better than chance
4. NN_scratch3 - change activation functions from ReLU to LeakyReLU
    - This helped very slightly

#### CNN models with transfer learning
 6. NN_trans - mobilenet2 pre-trained model
    - No better than chance
 7. NN_trans - resnet50 pre-trained model
    - No better than chance
    
### Second set of experiments


In [10]:
tensorboard = TensorBoard(log_dir = 'logs/NN_scratch')

In [11]:
metrics_ = [metrics.AUC(curve='ROC'), metrics.AUC(curve = 'PR'), metrics.Recall(), metrics.Precision(), metrics.PrecisionAtRecall(0.5), metrics.TruePositives(), metrics.TrueNegatives(), metrics.FalseNegatives(), metrics.FalsePositives()]
class_weights = {0:1, 1:15}

In [46]:
class_weights = {0:1, 1:15}
NN_scratch = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Flatten(),
                         Dense(32, activation='relu'),
                         Dropout(0.20),
                         Dense(1, activation='sigmoid')]
                       )

NN_scratch.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics = metrics_,
)


In [ ]:
NN_scratch = NN_scratch.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, class_weight = class_weights, validation_data = (X_valid, y_valid_array))

Epoch 1/10
156/156 [==============================] - 403s 3s/step - loss: 1.3569 - auc: 0.5076 - auc_1: 0.0704 - recall: 0.9523 - precision: 0.0690 - precision_at_recall: 0.0708 - true_positives: 1018.0000 - true_negatives: 787.0000 - false_negatives: 51.0000 - false_positives: 13742.0000 - val_loss: 0.7209 - val_auc: 0.4975 - val_auc_1: 0.0667 - val_recall: 1.0000 - val_precision: 0.0690 - val_precision_at_recall: 0.0696 - val_true_positives: 357.0000 - val_true_negatives: 24.0000 - val_false_negatives: 0.0000e+00 - val_false_positives: 4819.0000
Epoch 2/10
156/156 [==============================] - 376s 2s/step - loss: 1.3538 - auc: 0.5378 - auc_1: 0.0754 - recall: 0.9439 - precision: 0.0698 - precision_at_recall: 0.0764 - true_positives: 1009.0000 - true_negatives: 1090.0000 - false_negatives: 60.0000 - false_positives: 13439.0000 - val_loss: 0.7402 - val_auc: 0.4831 - val_auc_1: 0.0656 - val_recall: 0.9748 - val_precision: 0.0702 - val_precision_at_recall: 0.0707 - val_true_positi

In [14]:
results = NN_scratch.evaluate(X_test, y_test_cat)
print(results)

163/163 [==============================] - 36s 211ms/step - loss: 0.5069 - auc: 0.5156 - auc_1: 0.0735 - recall: 0.1039 - precision: 0.0774 - precision_at_recall: 0.0723 - true_positives: 37.0000 - true_negatives: 4403.0000 - false_negatives: 319.0000 - false_positives: 441.0000
[0.5068734288215637, 0.5155593156814575, 0.07354986667633057, 0.10393258184194565, 0.07740585505962372, 0.07227332144975662, 37.0, 4403.0, 319.0, 441.0]


In [15]:
accuracy_score(y_test_array, np.argmax(NN_scratch.predict(X_test), axis = -1))

0.9315384615384615

In [16]:
recall_score(y_test_array, np.argmax(NN_scratch.predict(X_test), axis = -1))

0.0

In [17]:
roc_auc_score(y_test_array, np.argmax(NN_scratch.predict(X_test), axis = -1))

0.5

In [45]:
NN_scratch.history.history

{}

Add a convolution filter of 128 

In [26]:
import tensorflow as tf
conf_matrix = tf.math.confusion_matrix(labels=y_test_array,
                                       predictions=NN_scratch.predict(X_test))
conf_matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4844,    0],
       [ 356,    0]], dtype=int32)>

In [18]:
# Add convolution filter

NN_scratch1 = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Flatten(),
                         Dense(32, activation='relu'),
                         Dropout(0.20),
                         Dense(1, activation='sigmoid')]
                       )

NN_scratch1.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_,
)

In [19]:
NN_scratch1.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, validation_split=0.2, class_weight = class_weights, validation_data = (X_valid, y_valid_array), callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True)])

Epoch 1/10
 38/125 [========>.....................] - ETA: 4:12 - loss: 2.1853 - auc: 0.5073 - auc_1: 0.0710 - recall: 0.2110 - precision: 0.0678 - precision_at_recall: 0.0703 - true_positives: 105.2105 - true_negatives: 5194.3947 - false_negatives: 379.1316 - false_positives: 1471.2632

KeyboardInterrupt: 

In [ ]:
results = NN_scratch1.evaluate(X_test, y_test_array)
print(results)

In [ ]:
accuracy_score(y_test_array, np.argmax(NN_scratch1.predict(X_test), axis = -1))

In [ ]:
recall_score(y_test_array, np.argmax(NN_scratch1.predict(X_test), axis = -1))

In [ ]:
roc_auc_score(y_test_array, np.argmax(NN_scratch1.predict(X_test), axis = -1))

Add a dense layer of 16

In [ ]:
# Add dense layer 

NN_scratch2 = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
                         MaxPooling2D(),
                         Flatten(),
                         Dense(32, activation='relu'),
                         Dense(16, activation='relu'),
                         Dropout(0.20),
                         Dense(1, activation='sigmoid')]
                       )


NN_scratch2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_,
)

In [ ]:
NN_scratch2.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, class_weight = class_weights, validation_data = (X_valid, y_valid_array), callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True)])

In [ ]:
results = NN_scratch2.evaluate(X_test, y_test_array)
print(results)

In [ ]:
accuracy_score(y_test_array, np.argmax(NN_scratch2.predict(X_test), axis = -1))

In [ ]:
recall_score(y_test_array, np.argmax(NN_scratch2.predict(X_test), axis = -1))

In [ ]:
roc_auc_score(y_test_array, np.argmax(NN_scratch2.predict(X_test), axis = -1))

Change relu activation to leaky relu

In [ ]:
NN_scratch3 = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         Conv2D(filters=32, kernel_size=3, activation=LeakyReLU(alpha = 0.20), padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=64, kernel_size=3, activation=LeakyReLU(alpha = 0.20), padding='same'),
                         MaxPooling2D(),
                         Flatten(),
                         Dense(32, activation =  LeakyReLU(alpha=0.20)),
                         Dropout(0.20),
                         Dense(1, activation='sigmoid')]
                       )

NN_scratch3.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_,
)

In [ ]:
NN_scratch3.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, class_weight = class_weights, validation_data = (X_valid, y_valid_array))

In [ ]:
results = NN_scratch3.evaluate(X_test, y_test_array)
print(results)

In [ ]:
accuracy_score(y_test_array, np.argmax(NN_scratch3.predict(X_test), axis = -1))

In [ ]:
recall_score(y_test_array, np.argmax(NN_scratch3.predict(X_test), axis = -1))

In [ ]:
roc_auc_score(y_test_array, np.argmax(NN_scratch3.predict(X_test), axis = -1))

Set class weights to be balanced

In [ ]:
NN_scratch4 = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         Conv2D(filters=32, kernel_size=3, activation=LeakyReLU(alpha = 0.20), padding='same'),
                         MaxPooling2D(),
                         Conv2D(filters=64, kernel_size=3, activation=LeakyReLU(alpha = 0.20), padding='same'),
                         MaxPooling2D(),
                         Flatten(),
                         Dense(32, activation =  LeakyReLU(alpha=0.20)),            
                         Dense(1, activation='sigmoid')]
                       )

NN_scratch4.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_,
)

In [ ]:
results = NN_scratch4.evaluate(X_test, y_test_array)
print(results)

In [ ]:
accuracy_score(y_test_array, np.argmax(NN_scratch4.predict(X_test), axis = 1))

In [ ]:
recall_score(y_test_array, np.argmax(NN_scratch4.predict(X_test), axis = 1))

In [ ]:
roc_auc_score(y_test_array, np.argmax(NN_scratch4.predict(X_test), axis = 1))

NN_transfer - try out pre-trained models. First try MobilNetV2. Interestingly, MobileNet believes that fluorscent cell images are mostly objects found in the ocean. 

In [ ]:
model = mobilenet_v2.MobileNetV2(weights='imagenet',)

for img_ in image_array[10:15]:
    out = model.predict(img_)
    print('Predicted:', mobilenet_v2.decode_predictions(out))

Transfer learning with mobilenetv2

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs/NN_transfer')

In [ ]:
#exclude the final dense layers by setting include_top=False, and add new ones to train from scratch below
base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(228,228,3)) 

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    
    
# Establish new fully connected block
NN_transfer = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         ZeroPadding2D(padding=2),
                         base_model,
                         Flatten(),
                         Dense(32, activation='relu'),
                         Dropout(0.20),
                         Dense(1, activation='sigmoid')]
                       )

NN_transfer.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_)

In [ ]:
NN_transfer.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, validation_data = (X_valid, y_valid_array), class_weight = class_weights, callbacks = [tensorboard])

In [ ]:
results = NN_transfer.evaluate(X_test, y_test_array)
print(results)

In [ ]:
accuracy_score(y_test_array, np.argmax(NN_transfer.predict(X_test), axis = -1))

In [ ]:
recall_score(y_test_array, np.argmax(NN_transfer.predict(X_test), axis = -1))

In [ ]:
roc_auc_score(y_test_array, np.argmax(NN_transfer.predict(X_test), axis = -1))

In [ ]:
from matplotlib import pyplot as plt
history = NN_transfer.history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(bottom = 0)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('best_loss.png')

In [ ]:
from matplotlib import pyplot as plt
history = NN_transfer.history
plt.plot(history.history['auc_3'])
plt.plot(history.history['val_auc_3'])
plt.ylim(bottom = 0)
plt.title('model AUC-PR')
plt.ylabel('AUC-PR')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.tight_layout()
plt.savefig('best_AUC-PR.png')
plt.show()

In [ ]:
plot_model(
    NN_transfer,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

Now try resnet50 as pre-trained model, ended it because it was slow and not learning

In [ ]:
#exclude the final dense layers by setting include_top=False, and add new ones to train from scratch below
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(228,228,3)) 

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    
    
# Establish new fully connected block
NN_transfer1 = Sequential(
                        [InputLayer(input_shape=X_train.shape[1:]),
                         ZeroPadding2D(padding=2),
                         base_model,
                         Flatten(),
                         Dense(32, activation='relu'),
                         Dense(1, activation='sigmoid')]
                       )

NN_transfer1.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=metrics_)

In [ ]:
NN_transfer1.fit(X_train, y_train_array, epochs=10, verbose=1, batch_size = 100, validation_data = (X_valid, y_valid_array), class_weight=class_weights, callbacks=[tensorboard])

In [ ]:
accuracy_score(y_test, np.argmax(NN_transfer1.predict(X_test), axis = -1))

In [ ]:
recall_score(y_test, np.argmax(NN_transfer1.predict(X_test), axis = -1))

In [ ]:
roc_auc_score(y_test, np.argmax(NN_transfer1.predict(X_test), axis = -1))

In [ ]:
from matplotlib import pyplot as plt
history = NN_scratch3.history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(bottom = 0)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('best_loss.png')

In [ ]:
history.history

In [41]:
from matplotlib import pyplot as plt
# history = NN_scratch.history
plt.plot(NN_scratch.history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(bottom = 0)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('best_loss.png')

KeyError: 'loss'

In [43]:
NN_scratch.history.history.keys()

dict_keys([])